In [22]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random    

In [23]:
lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))    

In [24]:
pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))   

In [25]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")  

Training data created


C:\Users\patan\AppData\Local\Temp\ipykernel_69480\626267745.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [26]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax')) 

In [27]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])        

In [28]:
# fitting and saving the mode
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)      

Epoch 1/200
163/163 [==============================] - 1s 2ms/step - loss: 5.2074 - accuracy: 0.0098
Epoch 2/200
163/163 [==============================] - 0s 2ms/step - loss: 5.1063 - accuracy: 0.0356
Epoch 3/200
163/163 [==============================] - 0s 2ms/step - loss: 4.8875 - accuracy: 0.0381
Epoch 4/200
163/163 [==============================] - 0s 2ms/step - loss: 4.6515 - accuracy: 0.0467
Epoch 5/200
163/163 [==============================] - 0s 2ms/step - loss: 4.5284 - accuracy: 0.0602
Epoch 6/200
163/163 [==============================] - 0s 2ms/step - loss: 4.3308 - accuracy: 0.0897
Epoch 7/200
163/163 [==============================] - 0s 2ms/step - loss: 4.2128 - accuracy: 0.1118
Epoch 8/200
163/163 [==============================] - 0s 2ms/step - loss: 4.0751 - accuracy: 0.1204
Epoch 9/200
163/163 [==============================] - 0s 2ms/step - loss: 3.8853 - accuracy: 0.1671
Epoch 10/200
163/163 [==============================] - 0s 2ms/step - loss: 3.7750 - accura